In [73]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env


# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
# df = pd.concat([df.entry, df.entry.apply(parse_log_entry)], axis=1)
df = df.entry.apply(parse_log_entry)
                    
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

df = df.append(new)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


# Investigating Status Code

In [74]:
status_given_ip = (
    df.groupby('ip')
    .status_code.value_counts(normalize=True)
    .rename('proba_status_given_ip')
    .reset_index()
)
status_given_ip.head()

,ip,status_code,proba_status_given_ip
0,173.173.113.51,200,1.0
1,24.26.242.9,200,1.0
2,3.88.129.158,200,1.0
3,3.92.201.136,200,1.0
4,34.207.64.242,200,1.0


In [75]:
status_given_ip[status_given_ip.proba_status_given_ip != 1]

,ip,status_code,proba_status_given_ip
17,72.181.113.170,200,0.996737
18,72.181.113.170,499,0.003263
22,97.105.19.58,200,0.998833
23,97.105.19.58,499,0.001167


In [76]:
status_given_ip[status_given_ip.status_code != '200']

,ip,status_code,proba_status_given_ip
18,72.181.113.170,499,0.003263
19,95.31.16.121,301,1.000000
21,97.105.15.120,301,1.000000
23,97.105.19.58,499,0.001167


In [86]:
df[df.status_code == '301']

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
1,95.31.16.121,2019-04-17 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
2,97.105.15.120,2019-04-18 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0


In [100]:
df[df.status_code=='301']



,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
1,95.31.16.121,2019-04-17 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
2,97.105.15.120,2019-04-18 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0


In [95]:
df[df.status_code=='301']
df.timestamp = df.timestamp.astype('datetime64')

mask = df.apply(lambda x: True if x.timestamp.day in [16,17, 18, 19] else False, axis=1)
df.timestamp.dt.day.value_counts()

16    11947
17     2028
21        1
18        1
19        1
Name: timestamp, dtype: int64

In [98]:
later_than_301s = df[df.timestamp > pd.to_datetime('2019-04-17 19:36:41')]
earlier_than_301s = df[df.timestamp < pd.to_datetime('2019-04-17 19:36:41')]

earlier_than_301s

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,2019-04-16 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,2019-04-16 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,2019-04-16 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0
...,...,...,...,...,...,...,...,...
13969,97.105.19.58,2019-04-17 12:55:12,GET,/api/v1/sales?page=62,HTTP/1.1,200,511544,python-requests/2.21.0
13970,97.105.19.58,2019-04-17 12:55:12,GET,/api/v1/sales?page=65,HTTP/1.1,200,511201,python-requests/2.21.0
13971,97.105.19.58,2019-04-17 12:55:13,GET,/api/v1/sales?page=73,HTTP/1.1,200,511781,python-requests/2.21.0
13972,97.105.19.58,2019-04-17 12:55:13,GET,/api/v1/sales?page=69,HTTP/1.1,200,510136,python-requests/2.21.0


In [99]:
later_than_301s

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,95.31.18.119,2019-04-21 10:02:41,GET,/api/v1/items/,HTTP/1.1,200,1153005,python-requests/2.21.0
2,97.105.15.120,2019-04-18 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0
3,97.105.19.58,2019-04-19 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,200,2056327,python-requests/2.21.0


**Conclusion:** 
> * Two users got redirected, back to back.
> * Significant drop off in traffic after the 17th, when we got our first 301. Next day we get the second. Potential downtime during that period.
> * The last two days we get 200 statuses, indicating that whatever was wrong was fixed. Also used the same request paths, so server side problems were likely going on at the end of the 17th and at least most of the 18th.

In [77]:
status_given_ip.status_code.value_counts()

200    20
499     2
301     2
Name: status_code, dtype: int64

## Takeaways (status code):
* only two had abnormal responses, rest 200/okay
* 499 means client problem (problem with browser, app, program, etc)
* These two problems were on ip addresses that have made a lot of overall traffic. Isn't abnormal in any interesting sense.
* Nothing very notable about this set of info

# Investigating Request Methods

In [10]:
request_methods_ip = (
    df.groupby('ip')
    .request_method.value_counts(normalize=True)
    .rename('proba_method_given_ip')
    .reset_index()
)
request_methods_ip.head()

,ip,request_method,proba_method_given_ip
0,173.173.113.51,GET,1.0
1,24.26.242.9,GET,1.0
2,3.88.129.158,GET,1.0
3,3.92.201.136,GET,1.0
4,34.207.64.242,GET,1.0


In [11]:
request_methods_ip[request_methods_ip.proba_method_given_ip != 1]

,ip,request_method,proba_method_given_ip


 > **Conclusion 1:** All request methods per ip address were the same for each

In [15]:
request_methods_ip.request_method.value_counts()

GET    19
Name: request_method, dtype: int64

> **Conclusion 2:** All request methods were GET requests

## Takeaways (Request Methods):
> * All request methods were GET requests for every ip address
> * Nothing abnormal present

# Investigating User Agents:

In [27]:
df.user_agent.value_counts()

python-requests/2.21.0                                                                                                       12001
python-requests/2.20.1                                                                                                        1911
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36       34
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0                                               8
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                       7
Slackbot 1.0 (+https://api.slack.com/robots)                                                                                     6
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36        4
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gec

In [19]:
df.user_agent.nunique()

9

> **Conclusion 1:** 9 different user agents made all of the requests

In [35]:
second_most  = df.user_agent.value_counts().iloc[1]
user_counts = df.user_agent.value_counts().to_frame()

# percentage of users below two highest values to total count
user_counts[user_counts.user_agent < second_most].user_agent.sum()/ df.user_agent.value_counts().iloc[:2,].sum()

0.004456584243818286

> **Conclusion 2:** First two user agents make up 99.56% of requests

## Takeaways (User Agents):
> * 9 unique user agents
> * Two make up 99.56% of requests
> * Other 7 user agents had 200 responses, so nothing anomalous about them

# Investigating Request Size:

In [65]:
average_request_size = (
    df.groupby('ip')
    ['size'].mean()
    .rename('average_request_size')
)
size_by_ip = average_request_size.to_frame()
size_by_ip

,average_request_size
ip,
173.173.113.51,391622.779981
24.26.242.9,166264.666667
3.88.129.158,3561.000000
3.92.201.136,42.000000
34.207.64.242,162.000000
34.229.70.250,162.000000
35.174.209.2,162.000000
35.175.171.137,1801.500000
45.23.250.16,1328.000000


In [69]:
boop = df.ip.value_counts().to_frame()
size_by_ip = size_by_ip.join(boop).rename(columns={'ip': 'num_requests'})



In [70]:
zscore_request_size = ((size_by_ip.average_request_size - size_by_ip.average_request_size.mean()) / size_by_ip.average_request_size.std())
size_by_ip['zscore_request_size'] = zscore_request_size

In [71]:
size_by_ip

,average_request_size,zscore_request_size,num_requests
ip,,,
173.173.113.51,391622.779981,1.612827,1059
24.26.242.9,166264.666667,0.324528,21
3.88.129.158,3561.000000,-0.605596,1
3.92.201.136,42.000000,-0.625713,1
34.207.64.242,162.000000,-0.625027,1
34.229.70.250,162.000000,-0.625027,1
35.174.209.2,162.000000,-0.625027,1
35.175.171.137,1801.500000,-0.615654,2
45.23.250.16,1328.000000,-0.618361,1


**Conclusion(s):** 
> * **52.87.230.102** has a high average request size with only 2 requests. Might be worth noting, probably not a big deal.
> * **97.105.19.58** has the highest average request size and the most traffic, might indicate some scraping or potential routine server activity.

